## Objective 

There are 2 objectived 

1. Obtain the yaw pitch and roll 
2. Draw the vector thing 




# Import Dependencies 

In [5]:
from operator import rshift
import cv2 as cv 
import numpy as np
import mediapipe as mp 


import numpy as np
import time
import math


# Dependecines for CSV
import csv
import os

# for rotation matrix
from scipy.spatial.transform import Rotation as R

# Main code 

In [6]:

mp_face_mesh = mp.solutions.face_mesh
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ] 
LEFT_IRIS = [474,475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]
cap = cv.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as face_mesh:
    while True:
        ret, frame = cap.read()
        #print('frame :',frame)
        if not ret:
            break
        frame = cv.flip(frame, 1)
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        img_h, img_w = frame.shape[:2]
        results = face_mesh.process(rgb_frame)
        if results.multi_face_landmarks:
            # print(results.multi_face_landmarks[0].landmark)
            mesh_points=np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
            # print(mesh_points.shape)
            (l_cx, l_cy), l_radius = cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
            (r_cx, r_cy), r_radius = cv.minEnclosingCircle(mesh_points[RIGHT_IRIS])
            center_left = np.array([l_cx, l_cy], dtype=np.int32)
            center_right = np.array([r_cx, r_cy], dtype=np.int32)
            cv.circle(frame, center_left, int(l_radius), (255,0,255), 1, cv.LINE_AA)
            cv.circle(frame, center_right, int(r_radius), (255,0,255), 1, cv.LINE_AA)
            
            
            iris_3d = []
            iris_2d = []

            # For left iris 
            for face_landmarks in results.multi_face_landmarks:
                for idx, lm in enumerate(face_landmarks.landmark):
                    if idx == 474 or idx == 475 or idx == 476 or idx == 477 or idx == 469 or idx == 470 or idx == 471 or idx == 472:
                        
                        x, y = int(lm.x * img_w), int(lm.y * img_h)
                        
                        # Get the 2D Coordinates
                        iris_2d.append([x, y])
                        
                        
                        # Get the 3D Coordinate
                        iris_3d.append([x, y, lm.z])
                        
            
            
            # Convert it to the NumPy array
            iris_2d = np.array(iris_2d, dtype=np.float64)
            
            # Convert it to the NumPy array
            iris_3d = np.array(iris_3d, dtype=np.float64)
            
                
                        # The camera matrix
            focal_length = 1 * img_w

            cam_matrix = np.array([ [focal_length, 0, img_h / 2],
                                    [0, focal_length, img_w / 2],
                                    [0, 0, 1]])

            # The distortion parameters
            dist_matrix = np.zeros((4, 1), dtype=np.float64)

            # Solve PnP
            success, rot_vec, trans_vec = cv.solvePnP(iris_3d, iris_2d, cam_matrix, dist_matrix)

            # Get rotational matrix
            rmat, jac = cv.Rodrigues(rot_vec)
            
            
            r = R.from_matrix(rmat)
            #r = R.from_rotvec(np.transpose(rot_vec))
            
            angles = r.as_euler('zyx', degrees=True)
            angles = angles*1000
            print('angles',angles)
                    
            


            # Add the text on the image
            #cv2.putText(image, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
            cv.putText(frame, "t1_p: " + str(angles[2]), (50, 50), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv.putText(frame, "t1_y: " + str(angles[1]), (50, 100), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv.putText(frame, "t1_r: " + str(angles[0]), (50, 150), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            
        cv.imshow('img', frame)
        key = cv.waitKey(1)
        if key ==ord('q'):
            break
cap.release()
cv.destroyAllWindows()

angles [ 0.24224398 -6.02930191 -0.81602521]
angles [ 0.24548578 -5.86726512  0.03904677]
angles [ 0.2280431  -5.55100828 -0.23903742]
angles [ 0.23520104 -5.88493046 -0.42046934]
angles [ 0.25347859 -5.85613765 -1.16499236]
angles [ 0.25033485 -6.36250446 -0.60218262]
angles [ 0.24135921 -5.87528609 -0.25184195]
angles [ 0.23879137 -5.81275196 -0.24938266]
angles [ 0.23486903 -5.81344822 -0.7273771 ]
angles [ 0.25005463 -6.29851758 -0.79025711]
angles [ 0.23356614 -5.60828327 -0.54565994]
angles [ 0.24839082 -5.46840419 -0.86616756]
angles [ 0.24454166 -6.15545139 -0.3072436 ]
angles [ 0.20547342 -5.3278701  -0.45294469]
angles [ 0.23466037 -6.01158681 -0.23594142]
angles [ 0.22639708 -5.49267431 -0.45204579]
angles [ 0.24325806 -6.17852061 -0.2992498 ]
angles [ 0.24029051 -6.18151991 -0.48025481]
angles [ 0.25348631 -6.04909866 -0.61831154]
angles [ 0.23275896 -5.90334892 -0.57973636]
angles [ 0.19511248 -4.67835827 -0.53689805]
angles [ 0.22453567 -5.83982186 -0.49217019]
angles [ 0

angles [ 0.28775961 -7.2428562  -1.12643761]
angles [ 0.30696255 -6.99495444 -0.95913967]
angles [ 0.30269854 -7.45767961 -1.37874827]
angles [ 0.32385081 -8.13195481 -1.09286109]
angles [ 0.30812522 -7.67083142 -1.00347458]
angles [ 0.31984163 -7.60032975 -0.94338609]
angles [ 0.29362717 -7.50249336 -1.27431777]
angles [ 0.3342129  -7.94201751 -0.98430666]
angles [ 0.29634758 -6.93380218 -1.22189523]
angles [ 0.29427022 -6.88516816 -1.21351804]
angles [ 0.32398732 -8.19225532 -0.79415456]
angles [ 0.32018809 -7.4586504  -0.65033202]
angles [ 0.33616993 -7.87242156 -0.28007955]
angles [ 0.30539519 -7.47500166 -0.30084766]
angles [ 0.33156251 -8.14225421 -0.44914022]
angles [ 0.29530329 -7.00781836 -1.01016777]
angles [ 0.30522523 -6.9086283  -1.08934877]
angles [ 0.32089647 -7.62516778 -1.16711755]
angles [ 0.3103849  -7.64942541 -0.3417337 ]
angles [ 0.31363162 -7.81250346 -1.81705267]
angles [ 0.28695394 -7.4227463  -1.036147  ]
angles [ 0.31216952 -7.64401383 -1.01132449]
angles [ 0

angles [-0.03579232 -5.81271374 -0.07193318]
angles [-0.03223004 -6.13624503 -0.14026806]
angles [-0.03762595 -5.73493014  0.01097024]
angles [-0.03739807 -5.97278728 -0.08931912]
angles [-0.03922071 -6.03210695 -0.08323482]
angles [-0.03323195 -5.89107648 -0.07546394]
angles [-0.0357597  -5.91722932 -0.09276647]
angles [-0.03917109 -6.26906167 -0.0708247 ]
angles [-0.03603293 -6.28767421 -0.10199278]
angles [-0.03513549 -5.74275783 -0.10884698]
angles [-0.03600979 -5.95907015 -0.08152044]
angles [-0.0377019  -6.06235242 -0.11137785]
angles [-0.03619733 -6.05408771  0.02867378]
angles [-0.03494139 -5.76593257 -0.05652372]
angles [-0.03524226 -5.99757405 -0.06375289]
angles [-0.04004983 -6.22941529  0.01797442]
angles [-0.03783828 -6.34869278 -0.07004118]
angles [-0.03899722 -6.14208153 -0.10884919]
angles [-0.03468676 -5.80767612 -0.05471606]
angles [-0.03734817 -5.88147949 -0.1371402 ]
angles [-0.03669438 -6.20355643 -0.14301415]
angles [-0.03282551 -5.57859764  0.01562528]
angles [-0

angles [-0.01462453  0.50696998 -0.03377639]
angles [ 0.04708821 -1.59718306 -0.03789073]
angles [ 0.10332828 -3.48176349 -0.05253087]
angles [ 0.11066262 -3.61143645 -0.2487716 ]
angles [ 0.16408476 -4.66393998  0.21198889]
angles [ 0.21418479 -6.51714986 -0.01816999]
angles [ 0.22414061 -7.00121192 -0.83309454]
angles [ 0.29751828 -8.06487703 -0.39338118]
angles [ 0.24817919 -8.9001096  -0.73385292]
angles [  0.33056517 -10.26482535  -0.02126619]
angles [ 0.325633   -9.17317114 -0.97811732]
angles [  0.29383787 -10.26740659  -0.21362936]
angles [  0.37812926 -10.44482306  -1.22735416]
angles [  0.35800237 -11.58477821  -0.23216714]
angles [  0.40992109 -12.16064036  -0.45055251]
angles [  0.34075479 -12.68674875  -0.79376621]
angles [  0.4321028  -12.148779    -0.26020108]
angles [  0.51358667 -12.14172439  -1.60824702]
angles [  0.52929175 -13.11520867  -0.20065854]
angles [  0.57797495 -12.68253214  -1.06075   ]
angles [  0.68126456 -13.65320944  -1.08452144]
angles [  0.72639415 -

angles [  0.82894954 -20.49534608  -0.44738622]
angles [  0.81149188 -21.3865273   -0.48508052]
angles [ 8.32489480e-01 -2.04640707e+01 -1.69980873e-02]
angles [  0.80694713 -20.34117793  -1.04540241]
angles [  0.78357665 -20.8872921   -0.98139513]
angles [  0.79510459 -22.33949131   0.39806596]
angles [  0.71546534 -21.69024855   0.28213274]
angles [  0.80028921 -20.52452343  -1.29653873]
angles [  0.8083283  -20.62173898  -0.17911011]
angles [  0.82286741 -20.42618531  -0.19753328]
angles [  0.79909212 -20.17665261   0.21650035]
angles [  0.78753309 -20.41460149  -1.03575113]
angles [  0.78216701 -20.30190185  -1.51962098]
angles [  0.83713645 -20.82037311   0.20651129]
angles [  0.74439573 -20.09710066  -0.08089561]
angles [  0.78060931 -20.6443766   -1.20493695]
angles [  0.81397758 -20.32799328  -0.29444344]
angles [  0.81425874 -20.86130985   1.31132543]
angles [  0.77413787 -20.24962026  -1.19307152]
angles [  0.79910549 -20.3126457   -0.62921064]
angles [  0.81082853 -20.114757